# NOTE: Use the stocks using their tickers

In [69]:
# Importing the libraries
import pandas as pd
from pandas_datareader import get_data_yahoo
from datetime import datetime
import plotly.graph_objs as go
from plotly.offline import plot

In [28]:
class StockData:
    def __init__(self, ticker):
        self._ticker = ticker.upper()
        start = datetime(1900, 1, 1)
        end = datetime.today()
        
        try:
            # Reading data from pandas datareader
            self.stock_df = get_data_yahoo(self.ticker, start = start, end = end)
            
            # Writing the data into a CSV, in case API read fails
            self.stock_df.to_csv('Datasets/' + self.ticker + '.csv')
        except:
            self.stock_df = pd.read_csv('Datasets/' + self.ticker + '.csv', index_col = 'Date', parse_dates = True)
        
        # Start and end date of the stock's data
        self._start_date = self.stock_df.index[0]
        self._end_date = self.stock_df.index[-1]
        
    # Using property to get start date
    @property
    def start_date(self):
        return self._start_date
    
    # Using property to get end date
    @property
    def end_date(self):
        return self._end_date
    
    # Using property to get ticker
    @property
    def ticker(self):
        return self._ticker
    
    # Return data in date range
    def getDataInRange(self, start = None, end = None):
        # Handling start date
        if not start:
            start = self.start_date
        if start < self.start_date:
            start = self.start_date
        
        # Handling end date
        if not end:     
            end = self.end_date
        elif end > self.end_date:
            end = self.end_date
        
        return self.stock_df.loc[start:end].copy()
    
    # Plotting closing prices
    def plotClosingPrice(self, start = None, end = None, fig = None):     
        # Ranging DataFrame
        ranged_df = self.getDataInRange(start, end)
        
        # Data to plot
        data = go.Scatter(x = ranged_df.index, y = ranged_df['Close'], name = self.ticker)
        layout = go.Layout(xaxis = {'title': 'Date'},
                           yaxis = {'title': 'Price in $'},
                           hovermode = 'x', title = self.ticker,
                           xaxis_rangeslider_visible = True)
        fig.update_layout(title = None) if fig else None
        
        return go.Figure(data = data, layout = layout) if not fig else fig.add_trace(data)
    
    # Moving Windows
    def movingWindows(self, short_window = 40, long_window  = 100, **kwargs):
        '''
        Parameters
        ----------
        short_window: Fast moving window
        long_window: Slow moving window
        
        **kwargs
        --------
        grid: bool. Default is True
        start: datetime. Start date of plotting
        end: datetime. End date of plotting
        '''
    
        # Handling kwargs
        accepted_kwargs = ('grid', 'start', 'end')
        for key in kwargs:
            if key not in accepted_kwargs:
                raise ValueError(f"'{key}' is invalid key!\nAccepted keys are: {accepted_kwargs}")
        # Grid
        if 'grid' not in kwargs:
            kwargs['grid'] = True
        elif type(kwargs['grid']) != bool:
            raise TypeError("'type' of 'grid' must be bool.")
        # Start
        if 'start' not in kwargs:
            kwargs['start'] = self.start_date
        elif type(kwargs['start']) != datetime:
            raise TypeError("'type' of 'start' must be datetime.")
        # End
        if 'end' not in kwargs:
            kwargs['end'] = self.end_date
        elif type(kwargs['end']) != datetime:
            raise TypeError("'type' of 'end' must be datetime.")
            
        # Ranging DataFrame
        ranged_df = self.getDataInRange(start = kwargs['start'], end = kwargs['end'])

        # Short and long moving windows rolling mean
        ranged_df[f'{short_window} days'] = ranged_df['Adj Close'].rolling(window = short_window).mean()
        ranged_df[f'{long_window} days'] = ranged_df['Adj Close'].rolling(window = long_window).mean()
        
        # Plot adjusted close price, short and long windows rolling means
        layout = go.Layout(xaxis = {'showgrid': kwargs['grid'], 'title': 'Date'},
                           yaxis = {'showgrid': kwargs['grid'], 'title': 'Price in $'},
                           hovermode = 'x', title = self.ticker,
                           xaxis_rangeslider_visible = True)
        adj_trace = go.Scatter(x = ranged_df.index, y = ranged_df['Adj Close'], name = 'Adj Close')
        short_trace = go.Scatter(x = ranged_df.index, y = ranged_df[f'{short_window} days'], name = f'{short_window} days')
        long_trace = go.Scatter(x = ranged_df.index, y = ranged_df[f'{long_window} days'], name = f'{long_window} days')
        return go.Figure(data = [adj_trace, short_trace, long_trace], layout = layout)
    
    # Simple Moving Average - Crossover Strategy
    def SMA_CS(self, short_window = 40, long_window = 100, **kwargs):
        '''
        Parameters
        ----------
        short_window: Fast moving window
        long_window: Slow moving window
        
        **kwargs
        --------
        start: datetime. Start date of plotting
        end: datetime. End date of plotting
        '''
        
        # Ranged Data
        if 'start' not in kwargs:
            kwargs['start'] = self.start_date
        if 'end' not in kwargs:
            kwargs['end'] = self.end_date
        ranged_df = self.getDataInRange(start = kwargs['start'], end = kwargs['end'])
        
        # Initialize signals DataFrame with Signal column having values 0
        self.signals = pd.DataFrame(data = 0, index = ranged_df.index, columns = ['Signal'])

        # Create short and long moving averages columns
        self.signals[f'Short ({short_window} days)'] = ranged_df['Close'].rolling(window = short_window,
                                                                                  min_periods = 1,
                                                                                  center = False).mean()
        self.signals[f'Long ({long_window} days)'] = ranged_df['Close'].rolling(window = long_window,
                                                                           min_periods = 1,
                                                                           center = False).mean()

        # Create signals
        from numpy import where
        self.signals['Signal'][short_window:] = where(self.signals[f'Short ({short_window} days)'][short_window:] >
                                                 self.signals[f'Long ({long_window} days)'][short_window:], 1, 0)

        # Trading orders
        self.signals['Positions'] = self.signals['Signal'].diff()

        # Short and long moving averages
        short_avg = go.Scatter(x = self.signals.index, y = self.signals[f'Short ({short_window} days)'],
                                 name = f'Short ({short_window} days)')
        long_avg = go.Scatter(x = self.signals.index, y = self.signals[f'Long ({long_window} days)'],
                              name = f'Long ({long_window} days)')

        # Buy and sell signals
        size = 10
        buy_signal = go.Scatter(x = self.signals[self.signals['Positions'] == 1].index,
                                y = self.signals[self.signals['Positions'] == 1][f'Short ({short_window} days)'],
                                marker = {'symbol': 'triangle-up-dot', 'size': size, 'color': 'green'},
                                mode = 'markers', showlegend = False, hoverinfo = 'skip')
        sell_signal = go.Scatter(x = self.signals[self.signals['Positions'] == -1].index,
                                 y = self.signals[self.signals['Positions'] == -1][f'Short ({short_window} days)'],
                                 marker = {'symbol': 'triangle-down-dot', 'size': size, 'color': 'red'},
                                 mode = 'markers', showlegend = False, hoverinfo = 'skip')
        
        # Plotting SMAs
        layout = go.Layout(xaxis = {'title': 'Date'},
                           yaxis = {'title': 'Price in $'},
                           hovermode = 'x', title = self.ticker,
                           xaxis_rangeslider_visible = True)
        return go.Figure(data = [short_avg, long_avg, buy_signal, sell_signal], layout = layout)
        
    # Backtesting
    # Activate this function only after SMA_CS has been executed
    def backtest(self, initial_capital = 1000000, shares = 100):
        # Dataframe 'portfolio' to backtest SMA-CS
        try:
            self.portfolio = pd.DataFrame(index = self.signals.index)
        except AttributeError:
            print('First implement the crossover strategy')
            return

        # Buy 'shares' on the day when short moving average crosses long moving average
        self.portfolio[self.ticker] = shares * self.signals['Signal']

        # Differences in shares owned
        self.portfolio['Diff'] = self.portfolio[self.ticker].diff()

        # Holdings of the shares
        self.portfolio['Holdings'] = self.portfolio[self.ticker].multiply(self.stock_df['Adj Close'], axis = 0)

        # Cash in hand
        self.portfolio['Cash'] = initial_capital - self.portfolio['Diff'].multiply(self.stock_df['Adj Close']).cumsum()

        # Total at an instant
        self.portfolio['Total'] = self.portfolio['Holdings'] + self.portfolio['Cash']

        # Returns of stocks
        self.portfolio['Returns'] = self.portfolio['Total'].pct_change()
        
        # Plot total
        total = go.Scatter(x = self.portfolio.index, y = self.portfolio['Total'], name = 'Total')

        # Buy and sell signals
        size = 10
        buy_signal = go.Scatter(x = self.portfolio[self.signals['Positions'] == 1].index,
                                y = self.portfolio[self.signals['Positions'] == 1]['Total'],
                                marker = {'symbol': 'triangle-up-dot', 'size': size, 'color': 'green'},
                                mode = 'markers', showlegend = False, hoverinfo = 'skip')
        sell_signal = go.Scatter(x = self.portfolio[self.signals['Positions'] == -1].index,
                                 y = self.portfolio[self.signals['Positions'] == -1]['Total'],
                                 marker = {'symbol': 'triangle-down-dot', 'size': size, 'color': 'red'},
                                 mode = 'markers', showlegend = False, hoverinfo = 'skip')
        layout = go.Layout(xaxis = {'title': 'Date'},
                           yaxis = {'title': 'Price in $'},
                           hovermode = 'x', title = self.ticker,
                           xaxis_rangeslider_visible = True)
        return go.Figure(data = [total, buy_signal, sell_signal], layout = layout)

In [29]:
active_stocks = {}

In [30]:
def onCompare(request):
    ticker = request.GET.get('text', 'default')
    if ticker == 'default':
        return
    active_stocks[ticker] = StockData(ticker)
    start_dates = []
    end_dates = []
    for key in active_stocks:
        if key == 'fig':
            continue
        start_dates.append(active_stocks[key].start_date)
        end_dates.append(active_stocks[key].end_date)
        start = max(start_dates)
        end = min(end_dates)
    active_stocks['fig'] = active_stocks[ticker].plotClosingPrice(start = start, end = end,
                                                                  name = ticker, fig = active_stocks['fig'])
    plot(active_stocks['fig'], filename = 'Page.html', auto_open = False)
    return render(request, 'Compare.html')

In [31]:
def onSubmit(request):
    ticker = request.GET.get('text', 'default')
    if ticker == 'default':
        return
    active_stocks[ticker] = StockData(ticker)
    summary = active_stocks[ticker].summary()
    active_stocks['fig'] = active_stocks[ticker].plotClosingPrice(name = ticker)
    plot(active_stocks['fig'], filename = 'Page.html', auto_open = False)
    return render(request, 'Chart.html', {'summary': summary})

In [32]:
def onMovingWindows(request):
    short_window = request.GET.get('short_window', 'default')
    long_window = request.GET.get('long_window', 'default')
    if short_window == 'default' or long_window == 'default':
        return
    for key in active_stocks:
        if key != 'fig':
            plot(active_stocks[key].movingWindows(), filename = 'Page.html', auto_open = False)
    return render(request, 'MovingWindows.html')

In [33]:
def onSMA(request):
    short_window = request.GET.get('short_window', 'default')
    long_window = request.GET.get('long_window', 'default')
    if short_window == 'default' or long_window == 'default':
        return
    for key in active_stocks:
        if key != 'fig':
            plot(active_stocks[key].SMA_CS(), filename = 'Page.html', auto_open = False)
    return render(request, 'SMA.html')

In [34]:
def onRemoveMovingOrSMA(request):
    plot(active_stocks['fig'], filename = 'Page.html', auto_open = False)
    return render(request, 'Chart.html')

In [35]:
def onRemoveComparison(request):
    ticker = None # Pass ticker of button to remove
    active_stocks.pop(ticker)
    active_stocks['fig'].data = tuple(filter(lambda stock: stock.name != ticker, active_stocks['fig'].data))
    plot(active_stocks['fig'], filename = 'Page.html', auto_open = False)
    return render(request, 'Compare.html') if len(active_stocks['fig'].data) > 1 else render(request, 'Chart.html')

In [36]:
def summary(self):
    previous_close = self.stock_df.iloc[-2]['Close']
    today_open = self.stock_df.iloc[-1]['Open']
    all_time_max = self.stock_df['Adj Close'].max()
    all_time_min = self.stock_df['Adj Close'].min()
    today_pct_change = self.stock_df.iloc[-2:].pct_change().iloc[-1]['Adj Close']
    monthly_pct_change = self.stock_df.resample('M').mean().iloc[-2:].pct_change().iloc[-1]['Adj Close']

In [38]:
self = get_data_yahoo('AAPL')
self.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2009-12-31,30.478571,30.080000,30.447144,30.104286,88102700.0,26.372231
2010-01-04,30.642857,30.340000,30.490000,30.572857,123432400.0,26.782711
2010-01-05,30.798571,30.464285,30.657143,30.625713,150476200.0,26.829010
2010-01-06,30.747143,30.107143,30.625713,30.138571,138040000.0,26.402260
2010-01-07,30.285715,29.864286,30.250000,30.082857,119282800.0,26.353460
